# 读取带有时间列的数据

In [ ]:
# 指定了 parse_dates 参数来解析 TIMESTAMP 列中的日期数据。
ifms_anl = pd.read_csv('data.csv', parse_dates=['TIMESTAMP'])

# 查看基本信息

In [ ]:
def data_basic_information(df):

    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    print(f"数据行数: {df.shape[0]}")
    print(f"数据列数: {df.shape[1]}") 

    dtypes = df.dtypes.value_counts()
    print("\n数据类型分布:")
    for dtype, count in dtypes.items():
        print(f"- {dtype}: {count}列")

    print("\n列信息:")
    column_info = pd.DataFrame({
        '数据类型': df.dtypes,
        '非空值数量': df.count(),
        '空值数量': df.isnull().sum(),
        '空值百分比': (df.isnull().sum() / len(df) * 100).round(2),
        '唯一值数量': df.nunique(),
        '唯一值': {column: df[column].unique() for column in df.columns}
    })
    display(column_info)

    print("\n数值列的统计描述:")
    display(df.describe().T)

    cat_columns = df.select_dtypes(include=['object', 'category']).columns
    if not cat_columns.empty:
        print("\n分类列的统计描述:")
        cat_stats = pd.DataFrame({
            '唯一值数量': df[cat_columns].nunique(),
            '最常见值': [df[col].value_counts().index[0] if not df[col].value_counts().empty else None for col in cat_columns],
            '最常见值频次': [df[col].value_counts().iloc[0] if not df[col].value_counts().empty else 0 for col in cat_columns],
            '最常见值占比': [(df[col].value_counts().iloc[0] / df[col].count() * 100).round(2) if not df[col].value_counts().empty else 0 for col in cat_columns]
        })
        cat_stats['最常见值占比'] = cat_stats['最常见值占比'].astype(str) + '%'
        display(cat_stats)

# 转换二元数据为targe编码，保存为target列

In [ ]:
from sklearn.preprocessing import LabelEncoder

def target_encoded(df):
    le = LabelEncoder()
    df['target'] = le.fit_transform(df['acnc'])
    print("类别编码对应关系：")
    for i, label in enumerate(le.classes_):
        print(f"{label} -> {i}")
    return df

# 丢弃不用的列

In [ ]:
def drop_useless_columns(df):
    
    # 丢弃唯一值数量为 1 的列
    unique_counts = df.nunique()
    columns_to_drop = unique_counts[unique_counts == 1].index
    df = df.drop(columns=columns_to_drop)

    # 丢弃 全部为NaN 的列
    columns_to_drop = df.columns[df.isna().all()]
    df = df.drop(columns=columns_to_drop)

    # 丢弃object 类型的列
    object_columns = df.select_dtypes(include=['object']).columns
    df = df.drop(columns=object_columns)

    # 丢弃 datetime64[ns] 类型的列
    datetime_columns = df.select_dtypes(include=['datetime64[ns]']).columns
    df = df.drop(columns=datetime_columns)

    # 丢弃第 5 到第 11 列（索引从 0 开始，第 5 列的索引是 4，第 11 列的索引是 10）。
    df = df.drop(df.columns[4:11], axis=1)
    # 丢弃第 6 到第 10 列（索引从 0 开始，第 6 列的索引是 5，第 10 列的索引是 9）。
    df = df.drop(df.columns[5:10], axis=1)

    return df

# 根据相关性系数可视化相关性热力图（Correlation Heatmap）

相关性系数（Correlation Coefficient）的取值范围是[−1,1]，其含义如下：
* 1：完全正相关，表示两列的变化方向完全一致。
* -1：完全负相关，表示两列的变化方向完全相反。
* 0：无相关性，表示两列之间没有线性关系。

热力图的颜色
* 颜色深浅：颜色越深（通常为红色），表示相关性越强；颜色越浅（通常为蓝色），表示相关性越弱。
* 颜色方向：红色通常表示正相关，蓝色通常表示负相关。

对角线
* 热力图的对角线通常是深色的，因为每列与自身的相关性为 1。

解读步骤

* 强相关性：
寻找颜色较深的区域，这些区域对应的列之间相关性较强。
* 弱相关性：
寻找颜色较浅的区域，这些区域对应的列之间相关性较弱。
* 负相关性：
寻找蓝色区域，这些区域对应的列之间负相关。
* 无相关性：
寻找接近白色的区域，这些区域对应的列之间无相关性。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def correlation_heat_map(df):
    correlation_matrix = df.corr()
    # 根据结果适量调整图表大小
    plt.figure(figsize=(30, 30))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap')
    plt.show()

# 改变列名

In [ ]:
# 改变列名
df1 = df1.rename(columns={
    'plc0': 'c_pv_prewash', 
    'plc6': 'c_sp_prewash', 
    'plc1': 'c_pv_regeneration', 
    'plc2': 'c_pv_regeneration_oil_in', 
    'plc3': 'c_pv_regeneration_oil_out', 
    'plc5': 'mv_pv_prewash', 
    'plc4': 'c_pv_rinse',
    'plc7': 'c_sp_rinse',
    'plc8': 'pa_vacuum_dry',
    'plc9': 'kpa_prewash'
    })

# 保存为 CSV 文件
df1.to_csv('data/data_1prewash.csv', index=False)

# 标准化之后做PCA

In [ ]:
# 标准化数据

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.decomposition import PCA

# 初始化 PCA，设置降维后的维度
pca = PCA(n_components=2)  # 降维到 2 维
X_pca = pca.fit_transform(X_scaled)

In [ ]:
# 解释 PCA 结果
print("降维后的数据形状:", X_pca.shape)
print("每个主成分解释的方差比例:", pca.explained_variance_ratio_)
print("主成分的方向:\n", pca.components_)

# 可视化降维结果
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA Result')
plt.show()

# 显示结果的格式设置

In [ ]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [ ]:
# 禁用科学计数法
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# 保存为 CSV 文件
df1.to_csv('data/data_1prewash.csv', index=False)